<a href="https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text to Image
### Method: CLIP + Siren
This model combines an image classifier called **CLIP** from *OpenAI* and an image generator called **SIREN.**

The process works like this:
1.  The **SIREN** model at first generates a completely random image
2.  The **CLIP** image classifier looks at the image and compares it with the given text caption
3.  Using this feedback the weights of the **SIREN** model are adjusted slightly to better approximate the image caption
4.  These steps are repeated until we stop it manually
---
In order to run click the [Colab] link. And then select *Runtime -> Run all* from the menu bar.

[colab]: https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb

# Parameters

In [10]:
text = 'An illustration of a black and white cat'  #@param {type: "string"}
args = '--lower_bound_cutout 0.5 --image-width=384 --iterations=1000 --epochs=2 --save-every=2 --save-progress=True --open_folder=False --overwrite=True --save_video=True --seed=123'  #@param {type: "string"}

parameters = {
    "text": text,
    "args": args
}

# Code

In [6]:
!pip install deep-daze --upgrade
!pip install stomp.py imageio-ffmpeg asyncio websockets varname --upgrade

Requirement already up-to-date: deep-daze in /usr/local/lib/python3.7/dist-packages (0.10.2)
Requirement already up-to-date: stomp.py in /usr/local/lib/python3.7/dist-packages (7.0.0)
Requirement already up-to-date: imageio-ffmpeg in /usr/local/lib/python3.7/dist-packages (0.4.3)
Requirement already up-to-date: asyncio in /usr/local/lib/python3.7/dist-packages (3.4.3)
Requirement already up-to-date: websockets in /usr/local/lib/python3.7/dist-packages (8.1)
Requirement already up-to-date: varname in /usr/local/lib/python3.7/dist-packages (0.6.3)


In [16]:

#parameter_names = [varname(p) for p in parameters]
nameof(parameters[0])

NonVariableArgumentError: ignored

In [3]:
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import os
import stomp
import json
import itertools
from base64 import b64encode
from time import sleep
import requests
import os
from mimetypes import guess_type 
import re
flatten = itertools.chain.from_iterable

TEXT="A black and white technical illustration of merging a cat and a human through biohacking."
DEFAULT_PARAMS="--lower_bound_cutout 0.5 --image-width=384 --iterations=1000 --epochs=2 --save-every=2 --save-progress=True --open_folder=False --overwrite=True --save_video=True --seed=123"
GITHUB_REPO="voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb"
GITHUB_REPO_CLEAN = re.sub('[^0-9a-zA-Z]+', '_', GITHUB_REPO)
print(GITHUB_REPO_CLEAN)
conn = stomp.Connection([('b-4e89df1f-8226-4cc4-a518-4e1ac0023c97-1.mq.eu-central-1.amazonaws.com', 61614)], use_ssl=True)
conn.connect('guest', 'iamcolabguest', wait=True)
conn.subscribe(destination=f'/queue/deep-daze', id=1, ack='auto')

def run(text, params=DEFAULT_PARAMS, watch_media_paths=["./*.jpg","./*.mp4"]):
  print("Running",text)
  for to_remove in watch_media_paths:
    for filepath in glob(to_remove):
      print("Deleting",filepath,"before starting.")
      os.remove(filepath)

  cmd = f'imagine {params} "{text}"'
  
  p = Popen(cmd, stdout = PIPE, 
          stderr = STDOUT, shell = True)

  found_media_files=[]
  while True:
    line = p.stdout.readline()#
    if not line: break

    stdout.write(line)

    headers = { 
        "params": params,
        "text": text
    }
    conn.send(body=line.decode("utf-8"), headers=headers, destination='/topic/colabOut')
    
    new_found_media_files = get_matching_files(watch_media_paths)
    new_found_media_files.sort(key=os.path.getmtime)
    added_media_files = list(set(new_found_media_files)-set(found_media_files))
    found_media_files = new_found_media_files

    if len(added_media_files) > 0:
      for added in added_media_files:
        sleep(0.1)
        if len(added) > 0:
          filename = os.path.basename(added)
          mimetype, _ = guess_type(filename)
          print("Adding:", added,"type",mimetype)
          
          with open(added,"rb") as media_file:
            added_media = media_file.read()

            base64_encoded_media = f"data:{mimetype};base64,{b64encode(added_media).decode('ascii')}"
          
          headers = {
              **headers,         
              "type": mimetype,
              "filename": filename
              }
          conn.send(body=base64_encoded_media, headers=headers, destination='/topic/colabMediaOut')

def get_matching_files(search_paths):
  return list(flatten([glob(search_path) for search_path in search_paths]))


class MyListener(stomp.ConnectionListener):
    def __init__(self, conn,run):
        self.conn = conn
        self.run = run

    def on_error(self, frame):
        print('received an error "%s"' % frame.body)

    def on_message(self, frame):
        print('received a message "%s"' % frame.body)
        self.run(frame.body)
        print('processed message')

    def on_disconnected(self):
        print('disconnected')
        #connect_and_subscribe(self.conn)

conn.set_listener('', MyListener(conn,run))

#run(TEXT)
#conn.disconnect()
sleep(1000000)


Streaming output truncated to the last 5000 lines.
loss: -57.44:  75%|███████▌  | 753/1000 [11:24<03:44,  1.10it/s]
                                             
Adding: ./A_scientific_illustration_of_a_penis_penetrating_an_asshole_in_black_and_whit.000377.jpg type image/jpeg
loss: -55.78:  76%|███████▌  | 755/1000 [11:25<03:42,  1.10it/s]
                                             
Adding: ./A_scientific_illustration_of_a_penis_penetrating_an_asshole_in_black_and_whit.000378.jpg type image/jpeg
loss: -56.07:  76%|███████▌  | 757/1000 [11:27<03:40,  1.10it/s]
                                             
Adding: ./A_scientific_illustration_of_a_penis_penetrating_an_asshole_in_black_and_whit.000379.jpg type image/jpeg
loss: -56.37:  76%|███████▌  | 759/1000 [11:29<03:39,  1.10it/s]
                                             
Adding: ./A_scientific_illustration_of_a_penis_penetrating_an_asshole_in_black_and_whit.000380.jpg type image/jpeg
loss: -57.90:  76%|███████▌  | 761/1000 [11:3

KeyboardInterrupt: ignored

In [ ]:
conn.disconnect()

In [ ]:
!npm install -g --force-perms node-pre-gyp ipfs

npm WARN deprecated node-pre-gyp@0.17.0: Please upgrade to @mapbox/node-pre-gyp: the non-scoped node-pre-gyp package is deprecated and only the @mapbox scoped package will recieve updates in the future
npm WARN deprecated request@2.88.2: request has been deprecated, see https://github.com/request/request/issues/3142
npm WARN deprecated har-validator@5.1.5: this library is no longer supported
npm WARN deprecated debug@4.1.1: Debug versions >=3.2.0 <3.2.7 || >=4 <4.3.1 have a low-severity ReDos regression when used in a Node.js environment. It is recommended you upgrade to 3.2.7 or 4.3.1. (https://github.com/visionmedia/debug/issues/797)
